# Тестирование LLM-based NLI

NLI с использованием LLM для точного парсинга:
- k-NN поиск похожих примеров
- Few-shot промпт с примерами
- LLM парсинг (OpenRouter/VLLM/DeepSeek)
- JSON извлечение коннекторов

Требуется: OPENROUTER_API_KEY (или VLLM сервер)

In [7]:
import sys
from pathlib import Path
import os

grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

# Проверка API ключа
HAS_API_KEY = bool(os.getenv('OPENROUTER_API_KEY'))

print("Статус:")
print(f"  OPENROUTER_API_KEY: {'✓ Установлен' if HAS_API_KEY else '✗ Не установлен'}")

if not HAS_API_KEY:
    print("\n⚠ Для работы установите: set OPENROUTER_API_KEY=your-key")
    print("  Или используйте VLLM: backend='vllm'")

Статус:
  OPENROUTER_API_KEY: ✓ Установлен


## 1. Создание датасета примеров

In [8]:
from grapharchitect.services.nli.nli_dataset_item import NLIDatasetItem
from grapharchitect.services.embedding.simple_embedding_service import SimpleEmbeddingService
from grapharchitect.entities.connectors.task_representation import TaskRepresentation
from grapharchitect.entities.connectors.connector import Connector

# Embedding service для k-NN
embedding = SimpleEmbeddingService(dimension=384)

# Создаем примеры для few-shot
examples = []

# Пример 1: Классификация
rep1 = TaskRepresentation()
rep1.input_connector = Connector("text", "question")
rep1.output_connector = Connector("text", "category")

examples.append(NLIDatasetItem(
    task_text="Классифицировать текст по категориям",
    task_embedding=embedding.embed_text("Классифицировать текст"),
    representation=rep1
))

# Пример 2: QA
rep2 = TaskRepresentation()
rep2.input_connector = Connector("text", "question")
rep2.output_connector = Connector("text", "answer")

examples.append(NLIDatasetItem(
    task_text="Ответить на вопрос пользователя",
    task_embedding=embedding.embed_text("Ответить на вопрос"),
    representation=rep2
))

# Пример 3: Генерация
rep3 = TaskRepresentation()
rep3.input_connector = Connector("text", "outline")
rep3.output_connector = Connector("text", "content")

examples.append(NLIDatasetItem(
    task_text="Создать статью по плану",
    task_embedding=embedding.embed_text("Создать статью"),
    representation=rep3
))

print(f"Создано примеров: {len(examples)}")
for i, ex in enumerate(examples, 1):
    print(f"  {i}. {ex.task_text}")
    print(f"     {ex.representation.input_connector.format} → {ex.representation.output_connector.format}")

Создано примеров: 3
  1. Классифицировать текст по категориям
     text|question → text|category
  2. Ответить на вопрос пользователя
     text|question → text|answer
  3. Создать статью по плану
     text|outline → text|content


## 2. Инициализация LLM NLI

In [9]:
if HAS_API_KEY:
    from grapharchitect.services.nli.llm_nli_service import LLMNLIService
    
    # Создание LLM NLI с OpenRouter
    llm_nli = LLMNLIService(
        embedding_service=embedding,
        backend="openrouter",
        model_name="openai/gpt-3.5-turbo",
        k_similar=3,
        temperature=0.1  # Низкая для точности
    )
    
    # Загрузка датасета
    llm_nli.load_dataset(examples)
    
    print("✓ LLM NLI инициализирован")
    print(f"  Backend: OpenRouter")
    print(f"  Модель: gpt-3.5-turbo")
    print(f"  Датасет: {len(examples)} примеров")
else:
    print("⚠ Установите OPENROUTER_API_KEY для работы")
    llm_nli = None

INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:OpenRouter инициализирован: модель openai/gpt-3.5-turbo
INFO:grapharchitect.services.nli.llm_nli_service:OpenRouter backend initialized
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI initialized: openrouter with openai/gpt-3.5-turbo
INFO:grapharchitect.services.nli.llm_nli_service:Loaded 3 NLI examples


✓ LLM NLI инициализирован
  Backend: OpenRouter
  Модель: gpt-3.5-turbo
  Датасет: 3 примеров


## 3. Создание демо инструментов

In [10]:
from grapharchitect.entities.base_tool import BaseTool

class DemoTool(BaseTool):
    def __init__(self, name, input_fmt, output_fmt):
        super().__init__()
        self.metadata.tool_name = name
        
        inp = input_fmt.split("|")
        out = output_fmt.split("|")
        
        self.input = Connector(inp[0], inp[1])
        self.output = Connector(out[0], out[1])
    
    def execute(self, input_data):
        return "Success"

# Создаем набор инструментов
tools = [
    DemoTool("Classifier", "text|question", "text|category"),
    DemoTool("QA-System", "text|question", "text|answer"),
    DemoTool("Writer", "text|outline", "text|content"),
]

print(f"Создано инструментов: {len(tools)}")
for tool in tools:
    print(f"  {tool.metadata.tool_name}: {tool.input.format} → {tool.output.format}")

Создано инструментов: 3
  Classifier: text|question → text|category
  QA-System: text|question → text|answer
  Writer: text|outline → text|content


## 4. Тестирование парсинга

In [6]:
# Тестовые запросы
test_queries = [
    "Определить категорию этого текста",
    "Дать ответ на вопрос клиента",
    "Написать статью по плану"
]

if HAS_API_KEY and llm_nli:
    print("Тестирование LLM NLI:\n")
    
    for query in test_queries:
        print(f"Запрос: \"{query}\"")
        
        # Парсинг через LLM
        result = llm_nli.parse_task(query, tools, k=3)
        
        if result.success:
            rep = result.task_representation
            print(f"  Вход:  {rep.input_connector.format}")
            print(f"  Выход: {rep.output_connector.format}")
            print(f"  Уверенность: {result.confidence:.2f}")
            
            # Показываем использованные примеры
            if result.similar_examples:
                print(f"  Использовано примеров из k-NN: {len(result.similar_examples)}")
        else:
            print(f"  ✗ Ошибка: {result.error_message}")
        
        print()
else:
    print("Для теста установите OPENROUTER_API_KEY")
    print("\nОжидаемые результаты:")
    print("  1. 'Определить категорию' → text|question → text|category")
    print("  2. 'Дать ответ' → text|question → text|answer")
    print("  3. 'Написать статью' → text|outline → text|content")

Тестирование LLM NLI:

Запрос: "Определить категорию этого текста"


INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=347, completion=50, total=397
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI parsed: text|content → text|category


  Вход:  text|content
  Выход: text|category
  Уверенность: 0.85
  Использовано примеров из k-NN: 3

Запрос: "Дать ответ на вопрос клиента"


INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=344, completion=50, total=394
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI parsed: text|question → text|answer


  Вход:  text|question
  Выход: text|answer
  Уверенность: 0.85
  Использовано примеров из k-NN: 3

Запрос: "Написать статью по плану"


INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=347, completion=50, total=397
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI parsed: text|outline → text|content


  Вход:  text|outline
  Выход: text|content
  Уверенность: 0.85
  Использовано примеров из k-NN: 3



## 5. Сравнение k-NN vs LLM NLI

In [6]:
# k-NN NLI (базовый)
from grapharchitect.services.nli.natural_language_interface import NaturalLanguageInterface

knn_nli = NaturalLanguageInterface(embedding)
knn_nli.load_dataset(examples)

print("Сравнение k-NN NLI vs LLM NLI:\n")

test_query = "Категоризировать сообщение пользователя"

# k-NN подход
print(f"Запрос: \"{test_query}\"\n")

knn_result = knn_nli.parse_task(test_query, tools, k=3)

print("k-NN NLI:")
if knn_result.success:
    print(f"  {knn_result.task_representation.input_connector.format} → {knn_result.task_representation.output_connector.format}")
else:
    print(f"  Ошибка")

# LLM подход
if HAS_API_KEY and llm_nli:
    llm_result = llm_nli.parse_task(test_query, tools, k=3)
    
    print("\nLLM NLI:")
    if llm_result.success:
        print(f"  {llm_result.task_representation.input_connector.format} → {llm_result.task_representation.output_connector.format}")
    else:
        print(f"  Ошибка")
    
    # Сравнение
    if knn_result.success and llm_result.success:
        same = (knn_result.task_representation.input_connector.format == llm_result.task_representation.input_connector.format and
                knn_result.task_representation.output_connector.format == llm_result.task_representation.output_connector.format)
        
        print(f"\nРезультаты {'совпадают' if same else 'различаются'}")
else:
    print("\nLLM NLI: Требуется API ключ")

Сравнение k-NN NLI vs LLM NLI:

Запрос: "Категоризировать сообщение пользователя"

k-NN NLI:
  text|outline → text|content


INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=343, completion=50, total=393
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI parsed: text|question → text|category



LLM NLI:
  text|question → text|category

Результаты различаются


## 6. Интеграция с GraphArchitect

In [ ]:
if HAS_API_KEY and llm_nli:
    from grapharchitect.services.execution.execution_orchestrator import ExecutionOrchestrator
    from grapharchitect.services.selection.instrument_selector import InstrumentSelector
    from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
    from grapharchitect.services.pathfinding_algorithm import PathfindingAlgorithm
    from grapharchitect.entities.task_definition import TaskDefinition
    
    # Инициализация
    selector = InstrumentSelector(temperature_constant=1.0)
    finder = GraphStrategyFinder()
    orchestrator = ExecutionOrchestrator(embedding, selector, finder)
    
    # Генерация эмбеддингов для инструментов
    for tool in tools:
        tool.metadata.capabilities_embedding = embedding.embed_tool_capabilities(tool)
    
    # Задача на естественном языке
    natural_query = "Проклассифицировать этот отзыв клиента"
    
    print(f"Естественный запрос: \"{natural_query}\"\n")
    
    # LLM NLI парсинг
    nli_result = llm_nli.parse_task(natural_query, tools)
    
    if nli_result.success:
        rep = nli_result.task_representation
        
        print("LLM NLI распарсил:")
        print(f"  Вход: {rep.input_connector.format}")
        print(f"  Выход: {rep.output_connector.format}")
        
        # Создание задачи с распарсенными коннекторами
        task = TaskDefinition(
            description=natural_query,
            input_connector=rep.input_connector,
            output_connector=rep.output_connector,
            input_data="Отличный продукт!"
        )
        
        # Выполнение
        context = orchestrator.execute_task(task, tools, path_limit=2, top_k=2)
        #, algorithm=PathfindingAlgorithm.ANT_COLONY
        
        print(f"\nВыполнение:")
        print(f"  Статус: {context.status.value}")
        print(f"  Ошибка: {context.error_message}")
        print(f"  Результат: {context.result}")
        print(f"\n✓ Полный цикл: Естественный язык → NLI → GraphArchitect → Результат")
    else:
        print(f"✗ LLM NLI не смог распарсить: {nli_result.error_message}")
else:
    print("Для теста установите OPENROUTER_API_KEY")

Естественный запрос: "Проклассифицировать этот отзыв клиента"



INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=351, completion=50, total=401
INFO:grapharchitect.services.nli.llm_nli_service:LLM NLI parsed: text|content → text|category


LLM NLI распарсил:
  Вход: text|content
  Выход: text|category
start_format text|content text|category

Выполнение:
  Статус: failed
  Ошибка: Не найдено путей от входного к выходному коннектору
  Результат: None

✓ Полный цикл: Естественный язык → NLI → GraphArchitect → Результат


## Итоги

**LLM-based NLI реализован**:
- ✓ k-NN поиск похожих примеров
- ✓ Few-shot промпт (динамический)
- ✓ LLM парсинг (OpenRouter/VLLM/DeepSeek)
- ✓ JSON извлечение коннекторов
- ✓ Интеграция с GraphArchitect

**Преимущества vs k-NN**:
- Более точный парсинг (LLM понимает контекст)
- Работает на новых типах задач
- Few-shot адаптируется к запросу

**Преимущества vs fine-tuned модель**:
- Не требует дообучения
- Легко обновлять (новые примеры в датасет)
- Работает с любой LLM

**Применение**:
- Production NLI (с OpenRouter)
- Privacy (с VLLM локально)
- Высокая точность (с GPT-4)